In [ ]:
BASE_DIR = "/extra/ucinlp1/cbelem/experiments-apr-15/data"

TARGET_WORDS = [
    "muslim",
    "jewish",
    "buddhist",
    "christian",
    # 
    "doctor",
    "nurse",
    "mother",
    "father",
    # 
]

BASE_PATHS = [f"{BASE_DIR}/{t}.csv" for t in TARGET_WORDS]
BASE_DATA = [pd.read_csv(p, index_col=0) for p in BASE_PATHS]